In [ ]:
import newspaper
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup 
from newspaper import Article

stocks = {'Apple':'AAPL','Google':'GOOGL','facebook':'FB','Tesla':'TSLA','Microsoft':'MSFT','Yahoo':'AABA','Sage':'sage'}
list = []
def scrap(stock,days):
    date = 'true'
    i = 1
    day = (datetime.now() - timedelta(days=days-1)).strftime("%d")
    while date == 'true':
        url = 'http://www.nasdaq.com/symbol/'+stocks[stock]+'/news-headlines?page='+str(i)
        html = requests.get(url)
        soup = BeautifulSoup(html.text, "lxml")
        divs = soup.findAll('div',attrs={'class':'news-headlines'})
        spans = divs[0].findAll('span',attrs={'class':'fontS14px'})
        small = divs[0].findAll('small')
        i = i+1
        url_list = []
        for span in range(10):
            print("\nDate : ",small[span].text.strip().split('/')[1]+"\n")
            if(int(small[span].text.strip().split('/')[1]) < int(day)):
                date = 'false'
                break
            link = spans[span].find('a')
            print(link['href'])
            url_list.append(link['href'])
        for url in url_list:
            try:
                news = Article(url)
                news.download()
                time.sleep(1)
                news.parse()
            except:
                continue
            list.append({'Ticker':stock,'Title':news.title,'Text':news.text.replace('\n',' '),'Date':news.publish_date,'Author':news.authors})
    panda = pd.DataFrame(data=list,columns=['Ticker', 'Title', 'Text', 'Date', 'Author'])
    panda.to_csv(stock+'.txt', encoding='utf-8',index=False)
  
scrap('Sage',7)
